<a href="https://colab.research.google.com/github/Qagent/Qagent/blob/main/vernacular_dependency_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install dependencies

In [1]:
!python --version

Python 3.7.13


In [2]:
!python -m pip install "dask[complete]"
!pip install memory_profiler
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 837 kB 30.2 MB/s 
     |████████████████████████████████| 140 kB 79.5 MB/s 
     |████████████████████████████████| 830 kB 63.3 MB/s 
     |████████████████████████████████| 822 kB 55.3 MB/s 
     |████████████████████████████████| 802 kB 79.7 MB/s 
     |████████████████████████████████| 802 kB 62.7 MB/s 
     |████████████████████████████████| 793 kB 81.1 MB/s 
     |████████████████████████████████| 793 kB 73.6 MB/s 
     |████████████████████████████████| 791 kB 72.1 MB/s 
     |████████████████████████████████| 786 kB 72.5 MB/s 
     |████████████████████████████████| 779 kB 72.7 MB/s 
     |████████████████████████████████| 778 kB 67.9 MB/s 
     |████████████████████████████████| 776 kB 60.3 MB/s 
     |████████████████████████████████| 769 kB 71.9 MB/s 
     |████████████████████████████████| 766 kB 63.7 MB/s 
     |█████████████████████████

#Import Libraries


In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import sys
import os
from importlib import import_module
import numpy as np
import csv
from google.colab import drive
import pandas as pd 
import gc
import itertools as it
import collections
import re
import dask as dask
import dask.delayed as chainmap
import dask.dataframe as ditable
import dask.bag as diarray
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar
import io
import json
from collections import Counter
from memory_profiler import memory_usage
import memory_profiler
from dask.distributed import Client
#import pyarrow.dataset as pydas
%load_ext memory_profiler
from dask import delayed
import time

In [2]:
pbar = ProgressBar()
pbar.register()

#Mount Db

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Load data, clean text and save it to parquet files partitioned on language

##read_data_class

In [4]:
class read_data:
  
  def __init__(self,path):
    self.path = path
  
  def to_path(self):  
    x = self.path
    a = re.split('/',x)
    name = re.split('\.',a[-1])[0]
    d = a[:-2]
    d = [d for d in d if d != '']
    path = '/'+'/'.join(d)+'/'+'pre_processed_data_final'+'/'+name
    return path
  def read_csv(self):
    path = self.path
    data = ditable.read_csv(path,sep = 'rows',names = ['content'],header = None,dtype = str,assume_missing = True,engine='python' ,quotechar='"', on_bad_lines = 'skip', skip_blank_lines = True,quoting = csv.QUOTE_MINIMAL) 
    data = data.dropna(how ='any')
    def sanitary_check(x):
      x = x
      x = x.replace('<br>','.')
      x = x.replace('<b>',' ')
      x = x.replace('</b>',' ')
      x = x.replace('</u>',' ')
      x = x.replace('</i>',' ')
      x = x.replace('<u>',' ')
      x = x.replace('<i>',' ')
      x = x.replace('<i>',' ')
      x = re.sub(r'(\?){1,}','?',x)
      x = re.sub(r'(\.){1,}','.',x) 
      x = re.sub(r'(\n){1,}','\n',x)
      x = re.sub(r'(\|)+','|',x)
      x = re.sub(r'(\!+){1,}','!',x)
      x = re.sub(r'([\xa0])+',' ',x)
      new = re.sub(r',+',',',x) 
      new = re.sub(r'(\s)+'," ",new)
      new = re.sub(r'(^\s)+',"",new)
      return new
    data.content = data.content.apply(sanitary_check,meta=pd.Series(dtype = str))
    def len_fil(x):
      x = str(x)
      if x == None:
        return '<NA>'
      elif len(x)>75:
        return x
      else:
        return '<NA>'
    data.content = data.content.apply(len_fil)
    data = data[(data.content != '<br>') & (data.content != '<NA>') & (data.content != None)]  
    return data


##pre_process_class

In [5]:
class pre_process(read_data):

  def __init__(self,path):
    self.path = path        
    

  def load_dict(self):
    path = self.path
    di_path = char_counter_class(path).char_prob()  
    with open(di_path,'r',encoding = 'utf8') as f:
      dic = json.load(f)
    m = np.mean([dic[i+1][1] for i in range(len(dic)-1)])
    std =np.std([dic[i+1][1] for i in range(len(dic)-1)])
    score = m-std/100
    dic = dict(dic)
    dicti = {k:v for (k,v) in dic.items() if v > score}
    return dicti


  def clean_text(self):
    from_path = self.path
    data = read_data(from_path).read_csv() 
    to_path = read_data(from_path).to_path()
    dicti = self.load_dict()
    to_path = to_path+'/'+'parquet_files'
    
    def check_char(str_data):    
      str_data = str_data
      chk =[str_data[i] for i in range(len(str_data))]
      def filt(da):
        if da in dicti.keys():
          return da
        else: 
          return '[unkc]'
      new_a = list(map(filt,chk))
      num_unk = len([x for x in new_a if x == '[unkc]'])
      num_chars = len(new_a)
      if num_unk/num_chars > 0.2:
        new_a = ['missing']
      else:
        new_a = new_a

      new_a = "".join(new_a)
      return new_a
    def final_cleaning(x):
      x = x
      x = re.sub(r'(\s)+'," ",x)
      x = re.sub(r'(\.)+'," . ",x)
      x = re.sub(r'(\|)'," | ",x)
      x = re.sub(r'(\?)'," ? ",x)
      x = re.sub(r'(\!)'," ! ",x)
      x = re.sub(r'(\[unkc\])+',"[unkc]",x)
      return x
    def bow(a1):
      a = a1
      b = re.split('[(\s|,|\||\.|\?)]+',a)
      b = [b.lower().strip() for b in b if b != ''and b != ' 'and b != ' "" '] 
      data = b
      return data 
    def sentence(a1):
      a = a1
      b = re.split('[(|\||\.|\?)]+',a)
      b = [b.lower().strip() for b in b if b != ''and b != ' ' and b != ' "" '] 
      return b  
    data.content = data.content.apply(check_char,meta=pd.Series(dtype = str))
    data.content = data.content.apply(final_cleaning,meta=pd.Series(dtype = str))
    data = data[(data.content != '') & (data.content != None) & (data.content != 'missing')]
    data['bow'] = data.content.map(bow,meta = pd.Series(dtype = str))
    data['sentence'] = data.content.map(sentence,meta = pd.Series(dtype = str))
    data.to_parquet(to_path, engine = 'pyarrow',write_index = False,compute = False).compute(scheduler = 'distributed')
    gc.collect()



##char_counter_class

In [6]:
class char_counter_class(pre_process):
  def __init__(self,path):                      
    super().__init__(path)
    self.path = path  
  
  
  def char_prob(self):
    path = self.path
    to_path = read_data(path).to_path()
    to_path = to_path+'/'+'char_prob'
    fname = to_path+'/'+'char_prob.json'
    data = read_data(path).read_csv()
    data = data.repartition(npartitions = 24)
    data = data.persist(scheduler = 'distributed',optimize_graph = True)
    data_dic = data.content
    data_dic = data_dic.sample(frac = 0.4,random_state = 1)
    dict_data = diarray.from_sequence(data_dic).flatten().frequencies(sort = True).compute(scheduler = 'processes')  
    l = sum([dict_data[i][1 ]for i in range(len(dict_data))])
    prob = [dict_data[i][1]/l for i in range(len(dict_data))]
    dict_key = [dict_data[i][0] for i in range(len(dict_data))]
    dict_data = list(tuple(zip(dict_key,prob)))
    if not os.path.exists(to_path):
      os.makedirs(to_path)
    with open(fname, 'w', encoding='utf-8') as f:
      json.dump(dict_data, f, ensure_ascii = False)
    return fname



#Run Code

In [7]:
from dask.distributed import Client
client = Client(processes = True)
client

Client Scheduler: tcp://127.0.0.1:43497 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 54.76 GB


In [ ]:
%time eng = pre_process('/content/drive/My Drive/DataNew/prenglish.csv').clean_text()
%time od = pre_process('/content/drive/My Drive/DataNew/prodia.csv').clean_text()
%time ur = pre_process('/content/drive/My Drive/DataNew/prurdu.csv').clean_text()
%time pu = pre_process('/content/drive/My Drive/DataNew/prpunjabi.csv').clean_text()
%time be = pre_process('/content/drive/My Drive/DataNew/prbengali.csv').clean_text()
%time ta = pre_process('/content/drive/My Drive/DataNew/prtamil.csv').clean_text()
%time mar = pre_process('/content/drive/My Drive/DataNew/prmarathi.csv').clean_text()

[########################################] | 100% Completed |  4.8s
CPU times: user 15.6 s, sys: 3.03 s, total: 18.6 s
Wall time: 2min 52s
[########################################] | 100% Completed |  6.4s
CPU times: user 20.8 s, sys: 2.76 s, total: 23.5 s
Wall time: 3min 11s
[########################################] | 100% Completed |  3.3s
CPU times: user 10.4 s, sys: 1.77 s, total: 12.2 s
Wall time: 1min 57s
[########################################] | 100% Completed |  3.2s
CPU times: user 10.9 s, sys: 1.66 s, total: 12.6 s
Wall time: 1min 53s
[########################################] | 100% Completed | 44.7s
CPU times: user 2min 28s, sys: 22.1 s, total: 2min 51s
Wall time: 23min 23s
[########################################] | 100% Completed | 41.6s
CPU times: user 1min 56s, sys: 21.5 s, total: 2min 17s
Wall time: 19min 25s
[########################################] | 100% Completed | 36.7s
CPU times: user 1min 48s, sys: 16.9 s, total: 2min 5s
Wall time: 18min 9s


In [ ]:
%time hi = pre_process('/content/drive/My Drive/DataNew/prhindi.csv').clean_text()
%time mal = pre_process('/content/drive/My Drive/DataNew/prmalyali.csv').clean_text()

##Delete 0 byte files


In [ ]:
!find -name 'file*' -size 0 -delete